In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

cities = [
    'https://rentberry.com/apartments/s/austin-tx',
    'https://rentberry.com/apartments/s/buffalo-ny',
    'https://rentberry.com/apartments/s/dallas-tx',
    'https://rentberry.com/apartments/s/jersey-city-nj',
    'https://rentberry.com/apartments/s/las-vegas-nv',
    'https://rentberry.com/apartments/s/miami-fl',
    'https://rentberry.com/apartments/s/orlando-fl',
    'https://rentberry.com/ca/apartments/s/ottawa-on-canada',
    'https://rentberry.com/ca/apartments/s/vancouver-bc-canada',
    'https://rentberry.com/ca/apartments/s/toronto-on-canada',
    'https://rentberry.com/au/apartments/s/brisbane-qld-australia',
    'https://rentberry.com/au/apartments/s/melbourne-vic-australia',
    'https://rentberry.com/au/apartments/s/sydney-nsw-australia',
    'https://rentberry.com/jp/apartments/s/tokyo-japan',
    'https://rentberry.com/au/apartments/s/perth-wa-australia',
    'https://rentberry.com/nl/apartments/s/amsterdam-netherlands',
    'https://rentberry.com/gb/apartments/s/birmingham-uk',
    'https://rentberry.com/gb/apartments/s/glasgow-uk',
    'https://rentberry.com/de/apartments/s/frankfurt-germany',
    'https://rentberry.com/de/apartments/s/munich-germany',
]

driver = webdriver.Chrome()
def fetch_source_code(url):

    driver.get(url)

    source_code = driver.page_source
    return source_code


total_places = []
with open('addresses.txt', 'a') as file:
    for url in tqdm(cities):
        i = 0
        rent_places = []
        while True:
            i += 1
            source_code = fetch_source_code(url + f'?page={i}')
            soup = BeautifulSoup(source_code, 'html.parser')
            address_tags = soup.find_all("rb-property-card-address")
            if len(rent_places) == 300:
                break
            if address_tags:
                for tag in address_tags:
                    a_tag = tag.find("a")
                    if a_tag:
                        href = a_tag.get("href")
                        file.write(href + '\n')
                        rent_places.append(href)
                        print(len(rent_places))
                    if len(rent_places) == 300:
                        break
            else:
                print("Tags not found")
        total_places.append(rent_places)

print(len(total_places))

driver.quit()

In [ ]:
with open('addresses.txt', 'r') as file:
    addresses_content = file.readlines()

duplicate_indexes = {}

for i, address in enumerate(addresses_content):
    address = address.strip()
    if address in duplicate_indexes:
        duplicate_indexes[address].append(i)
    else:
        duplicate_indexes[address] = [i]

duplicate_indexes = {address: indexes for address, indexes in duplicate_indexes.items() if len(indexes) >= 2}

duplist = []
for address, indexes in duplicate_indexes.items():
    duplist.append(indexes[1])

new_array = [y for x, y in enumerate(addresses_content) if x not in duplist]

import pickle

with open('my_list.pkl', 'wb') as file:
    pickle.dump(new_array, file)

with open('my_list.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

In [ ]:
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
import pandas as pd
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")  # This line makes Chrome run in headless mode

countries = [
    ('austin', 'tx', 'usa'),
    ('buffalo', 'ny', 'usa'),
    ('dallas', 'tx', 'usa'),
    ('jersey city', 'nj', 'usa'),
    ('las vegas', 'nv', 'usa'),
    ('miami', 'fl', 'usa'),
    ('orlando', 'fl', 'usa'),
    ('ottawa', 'on', 'canada'),
    ('vancouver', 'bc', 'canada'),
    ('toronto', 'on', 'canada'),
    ('brisbane', 'qld', 'australia'),
    ('melbourne', 'vic', 'australia'),
    ('sydney', 'nsw', 'australia'),
    ('tokyo', 'none', 'japan'),
    ('perth', 'wa', 'australia'),
    ('amsterdam', 'none', 'netherlands'),
    ('birmingham', 'none', 'uk'),
    ('glasgow', 'none', 'uk'),
    ('frankfurt', 'he', 'germany'),
    ('munich', 'by', 'germany'),
]

duplicated_countries = [country for country in countries for _ in range(300)]

driver = webdriver.Chrome(options=chrome_options)
def fetch_source_code(url):

    driver.get(url)
    wait = WebDriverWait(driver, 5)

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "map-address")))
    except TimeoutException:
        print("Elements with class 'map-address' not found within the specified timeout. Skipping...")

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-value-container")))
    except TimeoutException:
        print("Elements with class 'price-value-container' not found within the specified timeout. Skipping...")

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "facilities-param")))
    except TimeoutException:
        print("Elements with class 'facilities-param-value' not found within the specified timeout. Skipping...")

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "amenities-section")))
    except TimeoutException:
        print("Elements with class 'facilities-param-value' not found within the specified timeout. Skipping...")

    source_code = driver.page_source

    return source_code

with open('my_list.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

df = pd.DataFrame()
pattern = r"/(\d+)-"
conversion_factor = 10.7639
for index, url in enumerate(loaded_list[:36]):

    match = re.search(pattern, url)

    if match:
        sno = match.group(1)
    else:
        sno = 'none'
    source_code = fetch_source_code(url.strip())
    soup = BeautifulSoup(source_code, 'html.parser')

    try:
        address = soup.find(class_="map-address").text
    except:
        address = 'none'

    try:
        rent = soup.find(class_="price-value-container").text
    except:
        rent = 'none'

    try:
        facilities = soup.find_all(class_="facilities-param")
        facilities = [(x.text.strip().split('\n')[0], x.text.strip().split('\n')[1]) for x in facilities]
    except:
        facilities = []

    try:
        deposit = [x for x, y in facilities if 'Deposit' in y][0]
    except:
        deposit = 'none'

    try:
        bed = float([x for x, y in facilities if 'Bed' in y][0].replace(',', ''))
    except:
        bed = -1

    try:
        bath = float([x for x, y in facilities if 'Bath' in y][0].replace(',', ''))
    except:
        bath = -1

    try:
        pets = ','.join([x for x, y in facilities if 'Allowed' in y])
    except:
        pets = 'none'

    try:
        amnities = [x.find(class_='label').text for x in soup.find_all(class_='amenity-wrapper')]
        amnities.extend([x.text for x in soup.find_all(class_='amenity-wrapper')])
    except:
        amnities = []
    washer_dryer = 'Washer/Dryer In Unit' in amnities
    ac = 'Air Conditioning' in amnities
    park = 'Parking' in amnities
    fireplace = 'Fireplace' in amnities
    dishwasher = 'Dishwasher' in amnities
    floor = 'Hardwood Floors' in amnities
    deck = 'Roof Deck' in amnities
    storage = 'Storage' in amnities
    try:
        area = float([x for x, y in facilities if 'Sq M' in y][0].replace(",", "")) * conversion_factor
    except:
        area = float([x for x, y in facilities if 'Sq Ft' in y][0].replace(",", ""))

    city, state, country = duplicated_countries[:36][index]
    data_dict = {
        'Sno': sno,
        'Address': address.replace('\n', ''),
        'City': city,
        'State code': state,
        'Country': country,
        'Rental Price': rent.replace('\n', '').replace(' ', ''),
        'Deposit': deposit.replace('\n', '').replace(' ', ''),
        'No of Bed': bed,
        'No of Bathroom': bath,
        'Area': area,
        'Pets Allowed': pets,
        'Washer/Dryer': washer_dryer,
        'AC': ac,
        'Parking': park,
        'Fireplace': fireplace,
        'Dishwasher': dishwasher,
        'Hardwood Floors': floor,
        'Roof Deck': deck,
        'Storage': storage
    }

    print(index)
    print(data_dict)

    df_dictionary = pd.DataFrame([data_dict])
    df = pd.concat([df, df_dictionary], ignore_index=True)
    df.to_csv('data.csv')


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("data.csv", index_col='index')

df=df.drop_duplicates('Sno')
df = df[df['Area'] != -1]
df['Rental Price']=df['Rental Price'].str.replace(r'\*\*USD.*', '', regex=True)
to_process = [(b[0], b[1]) for a,b in df[df['City'] == 'tokyo'][df['Rental Price'].str.contains('M')][['Sno', 'Rental Price']].iterrows()]
for sno, price in to_process:
  df.loc[df['Sno'] == sno, 'Rental Price']=round(float(price.split('*')[0].replace('¥', '').replace(',', '').replace('M', '')) * 1000000 * 0.55)
df['Rental Price']=df['Rental Price'].str.replace(r'[^0-9.]', '', regex=True)
df['Rental Price']=df['Rental Price'].astype('int')

df['Deposit'] = df['Deposit'].str.replace(r'[^0-9.]', '', regex=True)
df['Deposit'] = df['Deposit'].astype('int')
df['Deposit'].replace('', np.nan, inplace=True)

df.replace(-1, np.nan, inplace=True)
df.replace('none', np.nan, inplace=True)

exchange_rates_to_INR = {
    'brisbane': 54.33, 'melbourne': 54.33, 'perth': 54.33, 'sydney': 54.33,
    'vancouver': 61.46, 'toronto': 61.46, 'ottawa': 61.46,
    'frankfurt': 90.73, 'munich': 90.73,
    'tokyo': 0.55,
    'amsterdam': 90.73,
    'glasgow': 105.10, 'birmingham': 105.10,
    'las vegas': 83.12, 'miami': 83.12, 'jersey city': 83.12, 'orlando': 83.12, 'dallas': 83.12, 'buffalo': 83.12, 'austin': 83.12
}
df['Exchange Rate'] = df['City'].map(exchange_rates_to_INR)
df['Rental Price INR'] = df['Rental Price'] * df['Exchange Rate']
df['Deposit INR'] = df['Deposit'] * df['Exchange Rate']
df['Deposit']=df['Deposit INR']
df.drop(columns=['Deposit INR'], inplace=True)
df.drop(columns=['Exchange Rate'], inplace=True)
df['Rental Price'] = df['Rental Price INR']
df.drop(columns=['Rental Price INR'], inplace=True)

columns_to_replace = ['Washer/Dryer', 'AC', 'Parking', 'Fireplace', 'Dishwasher',
                      'Hardwood Floors', 'Roof Deck', 'Storage']
df[columns_to_replace] = df[columns_to_replace].replace({True: 'Yes', False: 'No'})

df['Pets Allowed'] = df['Pets Allowed'].replace('Dogs,Cats', 'Yes')
df['Pets Allowed'] = df['Pets Allowed'].replace('Cats', 'Yes')
df['Pets Allowed'] = df['Pets Allowed'].replace('Dogs', 'Yes')
df['Pets Allowed'] = df['Pets Allowed'].replace('No Pets Allowed', 'No')

df['No of Bed'].corr(df['No of Bathroom'])
average_ratio_bed_to_bathroom = (df['No of Bed'] / df['No of Bathroom']).mean()
average_ratio_bathroom_to_bed = (df['No of Bathroom'] / df['No of Bed']).mean()

df['No of Bed'].fillna(df['No of Bathroom'] * average_ratio_bathroom_to_bed, inplace=True)
df['No of Bathroom'].fillna(df['No of Bed'] * average_ratio_bed_to_bathroom, inplace=True)

df['No of Bathroom'].fillna(df['No of Bathroom'].mean(), inplace=True)
df['No of Bed'].fillna(df['No of Bed'].mean(), inplace=True)
df['No of Bed'] = df['No of Bed'].round().astype(int)
df['No of Bathroom'] = df['No of Bathroom'].round().astype(int)

mode_pets_allowed_by_city = df.groupby('City')['Pets Allowed'].transform(lambda x: x.mode().iloc[0])
df['Pets Allowed'].fillna(mode_pets_allowed_by_city, inplace=True)

columns_to_fill = ['Washer/Dryer', 'AC', 'Parking', 'Fireplace', 'Dishwasher', 'Hardwood Floors', 'Roof Deck', 'Storage']

for column in columns_to_fill:
    df[column].fillna(df[column].mode().iloc[0], inplace=True)

In [ ]:
df.to_csv('newest_processed.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('newest_processed.csv')
column1 = df[['Sno','Address','City','State code','Country','Rental Price','Deposit']]
column2 = df[['Sno', 'No of Bed','No of Bathroom','Area','Pets Allowed']]
column3 = df[['Sno', 'Washer/Dryer','AC','Parking','Fireplace','Dishwasher','Hardwood Floors','Roof Deck','Storage']]

column1.to_csv('Table1.csv', index=False)
column2.to_csv('Table2.csv', index=False)
column3.to_csv('Table3.csv', index=False)